In this notebook, we introduce a tensor-based implementation of the HOG feature extractor, built in Keras and Tensorflow. It is, to our knowledge, the first of its kind. To restrict all operations to the tensor domain, we approximate the previously employed histogram bin construction methodology, known as voting via bi-linear interpolation, as the scalar projection of each angle vector onto each separate bin unit vector. Since this is defined as a simple dot product between the two aforementioned vectors, we can implement this as a Convolution2D operation, with the angle vectors as the input and the bin unit vectors as the filters, with each filter representing a single unit vector that operates on every angle vector in the image. A Relu activation function operates on the output to curb the influence of negative projections. The reshaping required for the bin operations broken into separate depthwise convolution2D operations (via Tensorflow) followed via a Concatenation. 

As both Tensorflow allows the automatic computation of gradients, unlike the SkImage HOG, our Tensor-based HOG enables backpropagation through HOG. This opens up an array of previously unexplored possibilities. It is for instance now possible to have HOG as an intermediary layer, meaning that instead of having the raw images as inputs, we first pass them through a series of convolution layers and then into HOG, theoretically enabling the creation of a more powerful feature extractor. We shall test this premise in the not too distant future.

For this notebook, we simply compare the performance of the outputs of the flattened bin array on both a keras dense logictic regiression and a Scipy logistic regression. We use centered samples. The results show that a simple scipy logictic regression on the output predictions prerforms significantly better than the intergrated keras Dense layer. Our current aim is to correct this.

In [1]:
#Importing packages
import numpy as np
import keras 
from keras import backend as K
from keras.layers import Conv2D, Input, Dense, Lambda, multiply, AveragePooling2D, Concatenate, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.engine.topology import Layer
from keras.models import Model
from keras.utils import np_utils
from keras.models import load_model
import tensorflow as tf
from tensorflow.python.framework import function

import pickle
from sklearn.linear_model import SGDClassifier

import matplotlib.pyplot as plt
%matplotlib inline

K.set_image_dim_ordering('tf')
K.set_image_data_format('channels_last')

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
# Opening pre-configured training and testing data.
# Balanced classes and equal test/train split
# Order: Samples are balanced, shuffled, split into training and testing sets, and then fed into the data generator separately.
# Images are geneated through Keras' ImageDataGenerator, with the following settings:
# rotation_range=180, zoom_range=0.2, horizontal_flip=True, vertical_flip =True, fill_mode = "nearest")

file = open("train_y_VGG.dat",'rb')
train_y_n = pickle.load(file)
file.close()

file = open("test_y_VGG.dat",'rb')
test_y_n = pickle.load(file)
file.close()

file = open("train_x_VGG.dat",'rb')
train_x_n = pickle.load(file)
file.close()

file = open("test_x_VGG.dat",'rb')
test_x_n = pickle.load(file)
file.close()

In [3]:
seed = 76

In [4]:
train_x_n.shape

(6150, 384, 384)

In [5]:
# Defining Values
bins = 8         # number of bins in histogram
w = 2*np.pi/bins     # width of each bin
centers = np.arange(-np.pi, np.pi, w) + 0.5*w   #centers of each bin

cell_size = 8   # Each cell will contain cell_size^2 pixels
block_size = 2  # Each block will contain block_size^2 pixels
batch_size = 100

In [6]:
nb_train  = train_x_n.shape[0]
nb_test  = test_x_n.shape[0]

# Prosessing images to ensure there are no vanishing and exploding gradients
train_x_n += (10e-10)*np.random.randn(nb_train, 384, 384)
test_x_n += (10e-10)*np.random.randn(nb_test, 384, 384)

In [7]:
# Ensuring samples are divisible by batch size
if train_x_n.shape[0]%batch_size != 0:
    
    # Slicing off end of samples to ensure train_y.shape[0]%batch_size == 0
    train_y = train_y_n[:-(train_y_n.shape[0]%batch_size)]
    test_y = test_y_n[:-(test_y_n.shape[0]%batch_size)]
    
    # Slicing off end of samples to ensure train_x.shape[0]%batch_size == 0
    # Adding dimention for the colour chanels (It's greyscale, hence only 1)
    train_x = train_x_n.reshape((train_x_n.shape[0], train_x_n.shape[1], train_x_n.shape[2], 1))[:-(train_x_n.shape[0]%batch_size), :, :, :]
    test_x = test_x_n.reshape((test_x_n.shape[0], test_x_n.shape[1], test_x_n.shape[2], 1))[:-(test_x_n.shape[0]%batch_size), :, :, :]

# Special case (-0 indices returns zero images)
elif train_x_n.shape[0]%batch_size == 0:
    train_y = train_y_n
    test_y = test_y_n
    
    train_x = train_x_n.reshape((train_x_n.shape[0], train_x_n.shape[1], train_x_n.shape[2], 1))
    test_x = test_x_n.reshape((test_x_n.shape[0], test_x_n.shape[1], test_x_n.shape[2], 1))
    
else:
    print ("Biggly Error")

# Convert class vectors to binary class matrices (for Keras Dense)
conf_train_y = np_utils.to_categorical(train_y, 2)
conf_test_y = np_utils.to_categorical(test_y, 2)

In [8]:
conf_test_y.shape

(6100, 2)

In [9]:
# Redefining Gradient functions to fix vanishing and exploding gradients
@function.Defun(tf.float32, tf.float32)
def norm_grad(x, dy):
    return dy*(x/tf.norm(x))

@function.Defun(tf.float32, grad_func=norm_grad)
def norm(x):
    return tf.norm(x)

In [10]:
# Defining Weights for the vertical and horizontal convolutions to calculate the image gradients
prewitt_x = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]).reshape((1, 3, 3, 1, 1)) + 0.01*np.random.randn(1, 3, 3, 1, 1)
prewitt_y = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]]).reshape((1, 3, 3, 1, 1)) + 0.01*np.random.randn(1, 3, 3, 1, 1)

cent = np.vstack((np.sin(centers), np.cos(centers))).reshape((1, 1, 1, 2, 8))

In [11]:
prewitt_x

array([[[[[-0.99877557]],

         [[-0.00129192]],

         [[ 0.9788288 ]]],


        [[[-1.00167166]],

         [[-0.01058604]],

         [[ 0.99762672]]],


        [[[-0.99788489]],

         [[-0.00801211]],

         [[ 1.00165138]]]]])

In [12]:
# Generating Filters for the Bin Operations
def create_bin_filters(bin_dim):
    filters = np.zeros((bin_dim**2, bin_dim, bin_dim))

    count = 0 
    for i in range(bin_dim):
        for j in range(bin_dim):
            filters[count, i, j] = 1
            count += 1
    return filters   

In [13]:
# bin_dim must be 2
# Increasing this will require the adding of some more tf.nn.depthwise_conv2d functions. There is one for each element in a single filter (i.e bin_dim^2)

b_flt = create_bin_filters(2)
bin_filters_n = b_flt.reshape((b_flt.shape[0], b_flt.shape[1], b_flt.shape[2], 1)).repeat(bins, 3) # Simply copying each filter anlong the last axis to satisfy the required shape for weights array (see Tensorflow docs for tf.nn.depthwise_conv2d)

In [14]:
# Reshaping to satisfy required shape for weight array
bin_filters = bin_filters_n.reshape(bin_filters_n.shape[0], bin_filters_n.shape[1], bin_filters_n.shape[2], bin_filters_n.shape[3], 1).astype(np.float32)

In [15]:
bin_filters[0, :, :, 0, 0]

array([[1., 0.],
       [0., 0.]], dtype=float32)

In [16]:
prewitt_y.shape

(1, 3, 3, 1, 1)

In [17]:
def init_prewitt_x(shape, dtype=None):
    shp = prewitt_x.shape
    weight = K.variable(prewitt_x) + K.random_normal_variable(shp, mean=0.0, scale=0.001, dtype=dtype)
    return weight

def init_prewitt_y(shape, dtype=None):
    shp = prewitt_y.shape
    weight = K.variable(prewitt_y) + K.random_normal_variable(shp, mean=0.0, scale=0.001, dtype=dtype)
    return weight

In [18]:
# Model HyperParameters
cell_dim = 8
bin_dim = 2
bin_stride_length = 1
bs = bin_stride_length

# Number of cells along each dim 
cell_nb = 128//cell_dim
assert not 128 % cell_dim

In [19]:
filt1 = tf.convert_to_tensor(bin_filters[0, :, :, :, :])
filt2 = tf.convert_to_tensor(bin_filters[1, :, :, :, :])
filt3 = tf.convert_to_tensor(bin_filters[2, :, :, :, :])
filt4 = tf.convert_to_tensor(bin_filters[3, :, :, :, :])

In [20]:
def calculate_magnitudes(conv_stacked):
    #b_bound = 10.0e-10
    #b_fill = tf.fill(tf.shape(conv_stacked), tf.constant(b_bound))
    
    #mag = tf.where(tf.abs(conv_stacked) < b_bound, b_fill, conv_stacked)
    mags = tf.norm((conv_stacked), axis=3)
    return mags

def calculate_angles(conv_stacked):
    angles = tf.atan2(conv_stacked[:, :, :, 1], conv_stacked[:, :, :, 0])
    return angles

def calculate_sin_cos(angles):
    sin = K.sin(angles)
    cos = K.cos(angles)
    sin_cos = K.stack([sin, cos], axis =-1)
    return sin_cos
   
def bins1(cells):
    c_bins = tf.nn.depthwise_conv2d(cells, filt1, strides = (1, bs, bs, 1), padding="SAME")
    return c_bins

def bins2(cells):
    c_bins = tf.nn.depthwise_conv2d(cells, filt2, strides = (1, bs, bs, 1), padding="SAME")
    return c_bins 

def bins3(cells):
    c_bins = tf.nn.depthwise_conv2d(cells, filt3, strides = (1, bs, bs, 1), padding="SAME")
    return c_bins

def bins4(cells):
    c_bins = tf.nn.depthwise_conv2d(cells, filt4, strides = (1, bs, bs, 1), padding="SAME")
    return c_bins

def bin_norm(bins_layer):
    bins_norm = tf.div(bins_layer, tf.expand_dims(tf.sqrt(tf.norm(bins_layer, axis=-1)+0.00000001), -1))
    return bins_norm

def hog_norm(bins_layer):
    hog_norms = tf.div(bins_layer, tf.expand_dims(tf.sqrt(tf.norm(bins_layer, axis=-1)+0.00000001), -1))
    hog_norms_2 = tf.div(hog_norms, tf.expand_dims(tf.sqrt(tf.norm(hog_norms, axis=-1)+0.00000001), -1))
    return hog_norms_2

def bins_flatten(bins_norm):
    bins_flattened = tf.reshape(bins_norm, (batch_size, (cell_nb-bs)*(cell_nb-bs)*bins*(bin_dim**2)))
    return bins_flattened

def tensorflow_dense(bins_flattened):
    logistic = tf.layers.dense(inputs=bins_flattened, units = 2, activation="sigmoid")
    return logistic

"""
def bins_flatten(bins):
    bins_flatten = tf.reshape(bins, (-1))
    return bins_flatten

def bin_flatten_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 4  #only valid for 4D tensors
    shape = shape[0]*shape[1]*shape[2]*shape[3]
    return tuple(shape)
"""

'\ndef bins_flatten(bins):\n    bins_flatten = tf.reshape(bins, (-1))\n    return bins_flatten\n\ndef bin_flatten_output_shape(input_shape):\n    shape = list(input_shape)\n    assert len(shape) == 4  #only valid for 4D tensors\n    shape = shape[0]*shape[1]*shape[2]*shape[3]\n    return tuple(shape)\n'

In [21]:
flat_shape = batch_size*(cell_nb-bs)*(cell_nb-bs)*bins*(bin_dim**2)

In [22]:
flat_shape

720000

In [23]:
train_x.shape

(6100, 384, 384, 1)

In [26]:
# Building Model 
inputs = Input(shape = (384, 384, 1), name="input")

# Convolutions
x_conv = Conv2D(1, (3,3), strides=1, padding="same", data_format="channels_last", trainable=False, use_bias=False, name="conv_x")(inputs)
y_conv = Conv2D(1, (3,3), strides=1, padding="same", data_format="channels_last", trainable=False, use_bias=False, name="conv_y")(inputs)

# Stacking since it appears you cannot have more than a single input into a layer (i.e mags and angles)
conv_stacked = Concatenate(axis=-1, name="Conv_Stacked")([x_conv, y_conv])

# Calculating the gradient magnitudes and angles
mags = Lambda(calculate_magnitudes, output_shape=(384, 384), name="mags1")(conv_stacked)
mags = Lambda(lambda x: K.stack((mags, mags), axis=-1), name="mags2")(mags)   # To enable sin_cos_vec 
angles = Lambda(calculate_angles, output_shape=(384, 384), name="angles")(conv_stacked)

# Calculating the components of angles in the x and y direction
# Then multiplying by magnitudes, giving angle vectors
sin_cos = Lambda(calculate_sin_cos, output_shape=(384, 384, 2), name="sin_cos")(angles)
sin_cos_vec = multiply([sin_cos, mags], name="sin_cos_mag")

# Applying each filter (representing a single bin unit vector) to every angle vector in the image.
# Result is an array with shape (img_height, img_width, bins) where each bin is contains an angle vectors contribution to each bin
# Relu activation function to remove negative projections (represented by negative scalar dot products).
votes = Conv2D(8, kernel_size=(1, 1), strides=(1, 1), activation="relu", trainable=False, bias=False, name="votes")(sin_cos_vec) 

# A round about way of splitting the image (i.e vote array) into a bunch of non-overlapping cells of size (cell_size, cell_size)
# then concateting values at each bin level, giving shape of (cell_nb, cell_nb, bins)
# Result is an array of cells with histograms along the final axis
cells = AveragePooling2D(pool_size=cell_size, strides=cell_size, name="cells")(votes)
cells = Lambda(lambda x: x * (cell_size**2), name="cells2")(cells)

# Bin Operations
# Assuming that bin shape = (2, 2)
# A round about way of grouping the cells into overlapping blocks of 2 * 2 cells each. 
# Two horizontally or vertically consecutive blocks overlap by two cells, that is, the block strides. 
# As a consequence, each internal cell is covered by four blocks (if bin_dim=2).
bins1_layer = Lambda(bins1, trainable=False, name="bins1")(cells)
bins2_layer = Lambda(bins2, trainable=False, name="bins2")(cells)
bins3_layer = Lambda(bins3, trainable=False, name="bins3")(cells)
bins4_layer = Lambda(bins4, trainable=False, name="bins4")(cells)
bins_layer = Concatenate(axis=-1)([bins1_layer, bins2_layer, bins3_layer, bins4_layer])

# normalize each block feature by its Euclidean norm
bins_norm_layer = Lambda(bin_norm, name="norm1")(bins_layer)
hog_norm_layer = Lambda(hog_norm, name="norm2")(bins_norm_layer)
#bin_flatten_layer = Lambda(bins_flatten, name="flat")(hog_norm_layer)  # Flatten Resulting array

# Block 1
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1_')(hog_norm_layer)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2_')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool_')(x)
x = Dropout(0.05)(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1_')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2_')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool_')(x)
x = Dropout(0.075)(x)


# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1_')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2_')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3_')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4_')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool_')(x)
x = Dropout(0.1)(x)

# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(1, 1), name='block4_pool')(x)
x = Dropout(0.15)(x)

# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(1, 1), name='block5_pool')(x)
x = Dropout(0.2)(x)

# Classification block
x = Flatten(name='flatten')(x)
x = Dropout(0.2)(x)
x = Dense(200, activation='relu', name='fc1')(x)
x = Dropout(0.2)(x)
x = Dense(200, activation='relu', name='fc2_')(x)
x = Dropout(0.2)(x)

logistic_reg = Dense(2, activation = "softmax", trainable=True, name="lr_")(x)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, kernel_size=(1, 1), strides=(1, 1), activation="relu", trainable=False, name="votes", use_bias=False)`


In [27]:
model = Model(inputs=inputs, outputs=logistic_reg)
model.load_weights('model_VGG_max.h5', by_name=True)

In [ ]:
#model.load_weights('model_VGG_lite.h5', by_name=True)

In [ ]:
model_mags = Model(inputs=inputs, outputs=mags)
model_angles = Model(inputs=inputs, outputs=angles)
model_cells = Model(inputs=inputs, outputs=cells)
#model_flatten = Model(inputs=inputs, outputs=bin_flatten_layer)

In [ ]:
#np.asarray(model.layers[9].get_weights()).shape

In [ ]:
#model.layers[1].set_weights(prewitt_x)
#model.layers[2].set_weights(prewitt_y)


#model_mags.layers[1].set_weights(prewitt_x)
#model_mags.layers[2].set_weights(prewitt_y)

#model_angles.layers[1].set_weights(prewitt_x)
#model_angles.layers[2].set_weights(prewitt_y)

#model_cells.layers[1].set_weights(prewitt_x)
#model_cells.layers[2].set_weights(prewitt_y)

In [ ]:


#model.layers[9].set_weights(cent)
#model_cells.layers[9].set_weights(cent)

In [ ]:
img_nb = 0

In [ ]:
pred_mags = model_mags.predict(train_x[:batch_size, :, :, :], batch_size=batch_size)
pred_angles = model_angles.predict(train_x[:batch_size, :, :, :], batch_size=batch_size)
pred_cells = model_cells.predict(train_x[:batch_size, :, :, :], batch_size=batch_size)

In [ ]:
plt.imshow(pred_mags[img_nb, :, :, 0])
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(pred_angles[img_nb, :, :])
plt.colorbar()
plt.show()

In [ ]:
print (pred_angles.max())
print (pred_angles.min())

In [ ]:
bin_nbs = 1
plt.imshow(pred_cells[img_nb, :, :, bin_nbs])
plt.colorbar()
plt.show()

print (pred_cells[img_nb, :, :, bin_nbs].max())
print (pred_cells[img_nb, :, :, bin_nbs].min())

In [ ]:
model.summary()

In [ ]:
#model.layers[2].get_weights()

In [ ]:
adam = keras.optimizers.Adam(lr=0.001)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=["accuracy"])
history = model.fit(train_x, conf_train_y, epochs=5, batch_size=batch_size, validation_data=(test_x, conf_test_y))

Train on 6100 samples, validate on 6100 samples
Epoch 1/5
6100/6100 [==============================] - 61s 10ms/step - loss: 0.1064 - acc: 0.9734 - val_loss: 0.8423 - val_acc: 0.9066
Epoch 2/5
6100/6100 [==============================] - 59s 10ms/step - loss: 0.0705 - acc: 0.9810 - val_loss: 2.1031 - val_acc: 0.8233
Epoch 3/5
6000/6100 [============================>.] - ETA: 0s - loss: 0.0481 - acc: 0.9890

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model_VGG_max.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_VGG_max.h5")
print("Saved model to disk")

In [ ]:
np.asarray(model.layers[1].get_weights())[0, :, :, 0, 0]

In [ ]:
np.asarray(model.layers[2].get_weights())[0, :, :, 0, 0]

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid(True)
plt.show() 

In [ ]:
#train_HOG = model_flatten.predict(train_x, batch_size=batch_size)
#test_HOG = model_flatten.predict(test_x, batch_size=batch_size)
#print ("Done")

In [ ]:
# Logistic Regression
#clf = SGDClassifier(max_iter=1000)
#clf = clf.fit(train_HOG, train_y)

In [ ]:
#score = clf.score(test_HOG, test_y)
#score